In [ ]:
# https://www.geeksforgeeks.org/implementing-an-autoencoder-in-pytorch/
import torch
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt
import loader as load
import processor as pr
import config
from torch.utils.data import Dataset, DataLoader

import random
import numpy as np
import pandas as pd
import numpy as np
from skorch import NeuralNetRegressor
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, KFold
from sklearn.decomposition import NMF
import json


# torch.backends.cudnn.deterministic = True
# random.seed(1)
# torch.manual_seed(1)
# torch.cuda.manual_seed(1)
# np.random.seed(1)

NR_EXTRACTED_FEATURES = 30
NR_OUTER_HIDDEN_LAYERS = 100
CANCER = "STAD"
# LAYER = "aak_ge"
# LAYER = "tcma_gen"
LAYER = "tcma_gen_aak_ge"
NR_INPUT_FEATURES = len(config.modality_features[LAYER])

# CANCER = "COAD"
# CANCER = "HNSC"
# CANCER = "ESCA"
# CANCER = "READ"
CANCER_MODEL_PATH = f"{config.model_state_path}/{LAYER}/model_{CANCER.lower()}"
MODEL_PARAMETERS_PATH = f"{config.model_state_path}/{LAYER}/best_parameters_{CANCER.lower()}.json"

In [ ]:
# https://towardsdatascience.com/how-to-use-datasets-and-dataloader-in-pytorch-for-custom-text-data-270eed7f7c00

class OverlapDataset(Dataset):
    """Genus + GE dataset."""

    def __init__(self, layer, target, cancer):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        data, files_names = load.loadAll(includeStage=(target=="stage"), sameSamples=True, skipGenes=(layer!="aak_ge"), skipAE=True)
        layer_data = load.getSpecificData(layer, data, files_names)

        if target=="tumor":
            layer_data = load.attachTumorStatus(layer_data)
        else:
            layer_data = load.attachStageStatus(layer_data)

        if cancer != "all":
            x, y = pr.splitData(layer_data, target=target, project=cancer)
        else:
            layer_data = layer_data.drop(["project"], axis=1)
            x, y = pr.splitData(layer_data, target=target)

        # x = x.drop(x.iloc[:, 20:5201], axis=1)
        
        self.modality_features = x
        self.targets = y


    def __len__(self):
        return len(self.modality_features)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        sample_features = self.modality_features.iloc[idx].values
        sample_target = self.targets.iloc[idx]
        sample = {'features': sample_features, 'target': sample_target}

        return sample

In [ ]:
overlapped = OverlapDataset(LAYER, "tumor", CANCER)
# Display text and label.
print('\nFirst iteration of data set: ', next(iter(overlapped)), '\n')
# Print how many items are in the data set
print('Length of data set: ', len(overlapped), '\n')
# Print entire data set
print('Entire data set: ', list(DataLoader(overlapped))[:2], '\n')

# DataLoader is used to load the dataset
# for training
loader = torch.utils.data.DataLoader(dataset = overlapped,
									batch_size = 16,
									shuffle = True)
									
print('Batched data set: ', list(loader)[:2], '\n')

In [ ]:
# https://aacrjournals.org/clincancerres/article-pdf/24/6/1248/2049917/1248.pdf
# Chaudhary paper: architecture = 500 -> 100 -> 500
# Proportionately should be: 100 -> 30 -> 100
# Features: 35877 -> 5221
# Use further feature selection?
# Use more complex training and testing pipeline with data splitting?
class AE(torch.nn.Module):
	def __init__(self, input_features=5221, first_hidden_size=NR_OUTER_HIDDEN_LAYERS, second_hidden_size=0, hidden_features=30):
		super().__init__()
		
		# Building an linear encoder with Linear
		# layer followed by Relu activation function
		# 784 ==> 9
		encoder_layers = []
		encoder_layers.append(torch.nn.Linear(input_features, first_hidden_size))
		encoder_layers.append(torch.nn.ReLU())
		if second_hidden_size > 0:
			encoder_layers.append(torch.nn.Linear(first_hidden_size, second_hidden_size))
			encoder_layers.append(torch.nn.ReLU())
			encoder_layers.append(torch.nn.Linear(second_hidden_size, hidden_features))
		else:
			encoder_layers.append(torch.nn.Linear(first_hidden_size, hidden_features))
		# Otherwise try leaky ReLu And otherwise give up
		# encoder_layers.append(torch.nn.ReLU())
		self.encoder = torch.nn.Sequential(*encoder_layers)
		
		# Building an linear decoder with Linear
		# layer followed by Relu activation function
		# The Sigmoid activation function
		# outputs the value between 0 and 1
		# 9 ==> 784
		decoder_layers = []

		if second_hidden_size > 0:
			decoder_layers.append(torch.nn.Linear(hidden_features, second_hidden_size))
			decoder_layers.append(torch.nn.ReLU())
			decoder_layers.append(torch.nn.Linear(second_hidden_size, first_hidden_size))
		else:
			decoder_layers.append(torch.nn.Linear(hidden_features, first_hidden_size))

		decoder_layers.append(torch.nn.ReLU())

		
		decoder_layers.append(torch.nn.Linear(first_hidden_size, input_features))
		decoder_layers.append(torch.nn.Sigmoid())
		self.decoder = torch.nn.Sequential(*decoder_layers)

	def forward(self, x):
		encoded = self.encoder(x)
		decoded = self.decoder(encoded)
		# HAS TO RETURN DECODED AND THEN ENCODED AS SK PIPELINE WILL TAKE FIRST VALUE WHEN COMPUTING LOSS IN GRIDSEARCH
		return decoded, encoded

class AutoEncoderNet(NeuralNetRegressor):		
	def get_loss(self, y_pred, y_true, *args, **kwargs):
		decoded, encoded = y_pred
		return super().get_loss(decoded, y_true, *args, **kwargs)

In [ ]:
with open(MODEL_PARAMETERS_PATH) as f:
    parameters = json.load(f)
    
print(parameters)
best_epochs = parameters["general"]["max_epochs"]
# Model Initialization
# model = AE(input_features=NR_INPUT_FEATURES, hidden_features=NR_EXTRACTED_FEATURES)
model = AE(**parameters["module"])

# Validation using MSE Loss function
loss_function = torch.nn.MSELoss()

# Using an Adam Optimizer with lr = 0.1
optimizer = torch.optim.Adam(model.parameters(), **parameters["optimizer"])
# optimizer = torch.optim.Adam(model.parameters(),
# 							lr = 1e-2,
# 							weight_decay = 1e-8)

model

In [ ]:
overlapped = OverlapDataset(LAYER, "tumor", CANCER)

train_size = int(0.8 * len(overlapped))
test_size = len(overlapped) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(overlapped, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
									batch_size = 16,
									shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
									batch_size = 16,
									shuffle = True)

In [ ]:
print("last hidden item: ", len(test_loader), len(train_dataset), len(test_dataset))
for batch_id, batched_samples in enumerate(test_loader):
    print(len(batched_samples["features"]))

In [ ]:
epochs = best_epochs
outputs = []
train_losses = []
validation_losses = []
hidden_representation = None

for epoch in range(epochs):
	train_loss = 0.0
	valid_loss = 0.0

	model.train(True)
	for batch_id, batched_samples in enumerate(train_loader):
		features = batched_samples["features"].float()
		
		optimizer.zero_grad()

		# Output of Autoencoder
		reconstructed, hidden = model(features)
		
		# Calculating the loss function
		loss = loss_function(reconstructed, features)
		
		# The gradients are set to zero,
		# the gradient is computed and stored.
		# .step() performs parameter update
		loss.backward()

		optimizer.step()
		
		# Storing the train_losses in a list for plotting
		train_loss += loss.detach() * len(batched_samples["features"])
		outputs.append((epochs, features, reconstructed))
		hidden_representation = hidden[-1]

		print("last hidden item: ", hidden_representation)
		print("last hidden item: ", len(batched_samples), len(train_loader))
	

	model.train(False)
	for batch_id, batched_samples in enumerate(test_loader):
		features = batched_samples["features"].float()
		reconstructed, hidden = model(features)
		loss = loss_function(reconstructed, features)
		valid_loss += loss.detach() * len(batched_samples["features"])
	
	train_losses.append(train_loss / len(train_dataset))
	validation_losses.append(valid_loss / len(test_dataset))

In [ ]:

# Defining the Plot Style
plt.style.use('fivethirtyeight')
plt.xlabel('Epochs')
plt.ylabel('Loss')

# Plotting the last 100 values
plt.plot(train_losses[:], label="train_loss")
plt.plot(validation_losses[:], label="validation_loss")
plt.legend()

In [ ]:
torch.save(model.state_dict(), CANCER_MODEL_PATH)

In [ ]:
# https://machinelearningmastery.com/use-pytorch-deep-learning-models-with-scikit-learn/HYPE/
# https://colab.research.google.com/github/skorch-dev/skorch/blob/master/notebooks/Advanced_Usage.ipynb#scrollTo=GBsIqCiWsVCY

# https://skorch.readthedocs.io/en/stable/user/neuralnet.html
# https://github.com/skorch-dev/skorch/issues/451
# R PARAMETER TUNING

net = AutoEncoderNet(
    AE,
    max_epochs=10,
    criterion=torch.nn.MSELoss,
    optimizer=torch.optim.Adam,
    # optimizer__lr=0.1,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
    # Sets default split, not needed when using GridSearchCV
    train_split=None,
    # module__input_features=5221
    # lr=0.0001,
    # optimizer__weight_decay=1e-8,
    # max_epochs=30,
    # module__input_features=5221,
    # module__first_hidden_size=300,
    # module__second_hidden_size=0,
    # module__hidden_features=100,
)

base_params = {
    'optimizer__lr': [0.1, 0.01, 0.001, 0.0001][:],
    'optimizer__weight_decay': [1e-8][:],
    'max_epochs': [10, 20, 30, 40, 80][:],
    # 'module__input_features': [5221][:],
    'module__input_features': [NR_INPUT_FEATURES][:],
}
param_grid = [{
    **base_params,
    'module__first_hidden_size': [100, 200][:],
    'module__second_hidden_size': [0][:],
    'module__hidden_features': [50, 100][:],
    }, {
    **base_params,
    'module__first_hidden_size': [100, 200][:],
    'module__second_hidden_size': [0, 50][:],
    'module__hidden_features': [30][:],
    }, {
    **base_params,
    'module__first_hidden_size': [200][:],
    'module__second_hidden_size': [100][:],
    'module__hidden_features': [30, 50][:],
    }, {
    **base_params,
    'module__first_hidden_size': [100, 200][:],
    'module__second_hidden_size': [100][:],
    'module__hidden_features': [30, 50, 100][:],
    }, 
]

In [ ]:
# Test skorch wrapper model
overlapped = OverlapDataset(LAYER, "stage", CANCER)
X = torch.tensor(overlapped.modality_features.values, dtype=torch.float32)
y = torch.tensor(overlapped.targets, dtype=torch.float32)
net.fit(X, X)

In [ ]:
random_state = 42
# Implement custom stratified splitting, fix outputtiIng of score being nan (data related?), save model, save all scores for learning curve
inner_cv = KFold(n_splits=5, shuffle=True, random_state=random_state)

grid_search = GridSearchCV(
    estimator=net,
    param_grid=param_grid,
    cv=inner_cv,
    scoring="neg_root_mean_squared_error", #https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    refit=False,
    return_train_score=True,
    n_jobs=-1,
    # Debug nan values
    error_score="raise")

overlapped = OverlapDataset(LAYER, "stage", CANCER)
X = torch.tensor(overlapped.modality_features.values, dtype=torch.float32)
y = torch.tensor(overlapped.targets, dtype=torch.float32)
result = grid_search.fit(X, X)
# net.fit(X, X)

In [ ]:
# result.predict
result.cv_results_

In [ ]:
print("Best: %f using %s" % (result.best_score_, result.best_params_))
means = result.cv_results_['mean_test_score']
stds = result.cv_results_['std_test_score']
params = result.cv_results_['params']
print(means)
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
grouped_best_parameters = {"module":{}, "optimizer":{}, "general":{}}

for k, v in result.best_params_.items():
    if "module__" in k:
        grouped_best_parameters["module"][k[8:]] = v
    if "optimizer__" in k:
        grouped_best_parameters["optimizer"][k[11:]] = v
    else:
        grouped_best_parameters["general"][k] = v
grouped_best_parameters

load.createDirectory(MODEL_PARAMETERS_PATH)
with open(MODEL_PARAMETERS_PATH, "w+") as f:
    f.write(json.dumps(grouped_best_parameters))

In [ ]:

X = np.array([[1, 1], [2, 1], [3, 1.2], [4, 1], [5, 0.8], [6, 1]])
from sklearn.decomposition import NMF
import sklearn.metrics as metrics
model = NMF(n_components=2, init='random', random_state=0)
W = model.fit_transform(X)
H = model.components_
fit_model = model.fit(X)

In [ ]:
W

In [ ]:
def get_score(model, data, scorer=metrics.explained_variance_score):
    """ Estimate performance of the model on the data """
    prediction = model.inverse_transform(model.transform(data))
    return scorer(data, prediction)
print(get_score(fit_model, X))

In [ ]:
# Do we need to feature normalized data? https://compgenomr.github.io/book/matrix-factorization-methods-for-unsupervised-multi-omics-data-integration.html#joint-non-negative-matrix-factorization 
# 19 Components has 67 separate construction error
ks = [5, 20, 25, 30, 50]
perfs_train = []
perfs_test = []
x = overlapped.modality_features.values
for k in ks:
    nmf = NMF(n_components=k).fit(x)
    perfs_train.append(get_score(nmf, x))
    perfs_test.append(get_score(nmf, x))
    print(nmf.reconstruction_err_)
print(perfs_train)
print(perfs_test)

In [ ]:

tst = nmf.transform(x)
tst_or = overlapped.modality_features
b = tst_or.iloc[:,:0]
tst_df = pd.DataFrame(tst)
pd.concat([b, tst_df], axis=1, ignore_index=True)
# tst_df
result = b.copy(deep=True)
# for idx, row in b.iterrows():
for col_nr in range(tst.shape[1]):
    result[col_nr] = tst[:, col_nr] 
print(tst)
result


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# layer_name = "aak_ge"
# layer_name = "tcma_gen"
layer_name = "tcma_gen_aak_ge"
cancers = ["STAD"]
with torch.no_grad():

    def extractFeaturesAE(features, model):
        features_tensor = torch.tensor(features.values).float()
        _, hidden_representation = model(features_tensor)
        return pd.Series(hidden_representation)

    for target in config.prediction_targets[:]:
        data, files_names = load.loadAll(includeStage=(target=="stage"), sameSamples=True, skipGenes=(layer_name!="aak_ge"), skipAE=True)
        layer_data = load.getSpecificData(layer_name, data, files_names)
        
        # print(ge_genus)

        # do not attach the target status for stage or it will screw up the pipeline later on when it expects the raw target
        if target == "tumor":
            layer_data = load.attachTumorStatus(layer_data)

        x_integrateds = []
        
        for extraction_method in ["ae", "nmf"][:1]:
            # for c in ["COAD", "ESCA", "HNSC", "READ", "STAD"][:]:
            for c in cancers[:]:
                if c != "all":
                    x, y = pr.splitData(layer_data, target=target, project=c)
                else:
                    layer_data = layer_data.drop(["project"], axis=1)
                    x, y = pr.splitData(layer_data, target=target)

                nr_features_extracted = 0
                if extraction_method == "ae":
                    nr_features_extracted = len(config.modality_features[layer_name + "_ae"])
                    # nr_features_extracted = 30
                    best_param_path = f"{config.model_state_path}/{layer_name}/best_parameters_{c.lower()}.json"
                    with open(best_param_path) as f:
                        parameters = json.load(f)

                    model = AE(**parameters["module"])
                    current_cancer_model_path = f"{config.model_state_path}/{layer_name}/model_{c.lower()}"
                    # print(current_cancer_model_path)
                    model.load_state_dict(torch.load(current_cancer_model_path))
                    model.eval()
                
                    x_integrated = x.apply(extractFeaturesAE, model=model, axis=1)
                elif extraction_method == "nmf":
                    nr_features_extracted = 30
                    model = NMF(n_components=nr_features_extracted, init='random', random_state=0)
                    nmf = model.fit(x.values)
                    nmf_coef = nmf.transform(x.values)
                    x_integrated = x.iloc[:, :0]
                    for col_nr in range(nmf_coef.shape[1]):
                        x_integrated[col_nr] = nmf_coef[:, col_nr]
                x_integrated[target] = y
                x_integrated["project"] = c
                    
                print(c, x_integrated)
                x_integrateds.append(x_integrated)
            
            print(x_integrateds)
            integration_file_location = f"Data/Integration/{extraction_method}/{target}/{layer_name}"
            load.createDirectory(integration_file_location)
            ge_genus_integrated = pd.concat(x_integrateds)#, ignore_index=True)
            
            # Make sure final list has the same order as original list for to ensure training split happens the same every time
            if len(cancers) > 1:
                ge_genus_integrated = ge_genus_integrated.reindex(layer_data.index)
            else:
                # make the new index specific if there's only one cancer to avoid rows with missing values
                cancer_layer_data = layer_data[layer_data["project"]==cancers[0]]
                ge_genus_integrated = ge_genus_integrated.reindex(cancer_layer_data.index)
            # print(ge_genus_integrated)

            ge_genus_integrated_minmaxed = ge_genus_integrated.copy()
            latent_feature_names = ge_genus_integrated_minmaxed.columns[:nr_features_extracted]
            ge_genus_integrated_minmaxed[latent_feature_names] = scaler.fit_transform(ge_genus_integrated_minmaxed[latent_feature_names])

            ge_genus_integrated.to_csv(f"{integration_file_location}.csv")
            ge_genus_integrated_minmaxed.to_csv(f"{integration_file_location}_minmax.csv")

In [ ]:
config.modality_features[layer_name]